# Cost Modeling

In [2]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import sys, os
import numpy as np,pandas as pd
import datetime, time

base_path = os.path.dirname(os.getcwd())
sys.path.append(base_path)
src_path = base_path + "/src"
from src import utilities as util, CostAnalyzer

code_path, data_path, main_result_path = util.GetWorkingDirs(base_path) 
sys.path.append(base_path)
sys.path.append(code_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Select Target System: In case multiple systems in the result directory

In [3]:
# selecting target cost dataset working directory
system_name, data_path = util.GetTargetDir(main_result_path, "server name")
print("Target directory is: " + data_path)

The following systems are found in the result folder. Select server name :
1:System_CVS
2:System_LS
Target directory is: D:\GithubRepos\CCMT-NILM\code\results\System_CVS


## Cost Modeling: Data Prepration

In [4]:
# initialize CostModeler object
ObjModel = CostModeler.CostModeler(data_path=data_path, system_name=system_name)

# load cost datasets
df_cost, df_cost_agg = ObjModel.LoadWorkingDataset()

# preparing cost dataset for modeling
dataset, features, targets = ObjModel.FeatureDataPrepartation()

100%|██████████| 3/3 [00:00<00:00, 149.93it/s]costdataset size: (224, 31)



In [5]:
#  display the prepared modeling dataset
ObjModel.dataset.head()
ObjModel.dataset.shape

,Number of Houses,Number of Processes,dw_minutes,ProcTime_FromSystem,time_disagg_online_sec,ProcTime_FromMeter,ProcEnergy_FromMeter,meter_energy_bg_dw_wh,house_energy_min_kwh,house_energy_mean_kwh,house_energy_median_kwh,house_energy_max_kwh,house_energy_total_kwh
0,1000,1,60.0,561.4710,561.3935,556.5,7.02995,127.57030,0.0,0.38675,0.25635,4.03,386.74275
1,1000,2,60.0,228.4225,228.3210,227.0,5.26995,127.23700,0.0,0.38675,0.25635,4.03,386.74275
2,1000,3,60.0,154.0160,153.8760,154.0,4.17395,126.97000,0.0,0.38675,0.25635,4.03,386.74275
3,1000,4,60.0,120.8830,120.7345,121.0,3.39140,126.74380,0.0,0.38675,0.25635,4.03,386.74275
4,1000,5,60.0,103.1630,103.0170,104.0,2.94935,126.57105,0.0,0.38675,0.25635,4.03,386.74275


(64, 13)

## Cost Modeling: Development

### Cost: NILM Processing Time

In [6]:
# building cost model for processing time
target = 'ProcTime_FromMeter'
features = ObjModel.features

#### Hyperparameter Tuning

In [7]:
#  update the rf model parameters in the setting.JSON once the best paramters is identified
ObjMLModel = model.PredictiveModel(models_path=ObjModel.models_path, dataset=ObjModel.dataset, features=ObjModel.features, target=target)
param_grid = {
                            'n_estimators': np.arange(4, 10, 2),
                            'max_depth': np.arange(2, 10, 2),
                            # 'min_samples_leaf': np.arange(1, 5, 1),
                            # 'min_samples_split': np.arange(1, 5, 1),          
                            # 'max_leaf_nodes': list(np.arange(4, 20, 2)) + [None],
                            # 'bootstrap': [True, False],
                }

ObjMLModel.ParameterTuneGridCV(param_grid=param_grid, n_jobs=4)

K-fold cross-validation model hyper-parameter tuning using njobs =  4 ... 
features:  ['Number of Processes', 'num_houses_inv']
target:  ProcTime_FromMeter
unique groups based on Number of Houses: [1000 1500 2000 2500]
seed:  223
      mean_test_MAPE  mean_test_MdAPE  mean_test_APE_MAX  rank_test_MAPE  rank_test_MdAPE  rank_test_APE_MAX                                                    params
681        -1.342727        -0.712430          -4.447555               6                3                128  {'max_depth': 6, 'n_estimators': 4, 'random_state': 780}
697        -1.461541        -1.085525          -3.806322              48              235                  1  {'max_depth': 6, 'n_estimators': 4, 'random_state': 479}
830        -1.528269        -1.242682          -3.834189              82              456                  3  {'max_depth': 6, 'n_estimators': 8, 'random_state': 754}
881        -1.341194        -0.706506          -4.397701               4                2             

{}

#### Model building

In [10]:
print("Model building for {}...".format(target))
ObjCostModels_PT = ObjModel.ModelBuilding(features, target, isfigshow=False)

filename:  d:\GithubRepos\CCMT-NILM\code\src\setting.json
Model building for ProcTime_FromMeter...

Mean Estimator (ME) Model Fitting ...

features:  ['Number of Processes']
target:  ProcTime_FromMeter

****************************************
TEST_MSE_PUH: 4.9560698784722015e-06
TEST_MSE: 328.8784375000009
ALL_MAPE: 1.316
TRAIN_MAPE: 1.384
TEST_MAPE: 1.247
ALL_RVALUE: 1.0
TRAIN_RVALUE: 1.0
TEST_RVALUE: 1.0
ALL_APE_CI: [0.375, 0.984375, 1.0, 1.0]
APE_EXTREMES_ALL: {'MIN_APE': 0.03036, 'MAX_APE': 3.07932}
TRAIN_APE_CI: [0.3125, 0.96875, 1.0, 1.0]
APE_EXTREMES_TRAIN: {'MIN_APE': 0.03036, 'MAX_APE': 3.07932}
TEST_APE_CI: [0.4375, 1.0, 1.0, 1.0]
APE_EXTREMES_TEST: {'MIN_APE': 0.03901, 'MAX_APE': 2.97073}
APE_CI_THRESHOLDS: [1, 3, 5, 10]

****************************************

RandomForest Model Training ...

features:  ['Number of Processes', 'num_houses_inv']
target:  ProcTime_FromMeter
train seed:  223

****************************************
TEST_MSE_PUH: 5.6414248167438196e-06
TEST

### Save Model

In [11]:
ObjModel.ObjMLModel.costmodels
ObjModel.SaveTrainedModel(ObjCostModels_PT)

{'ME': {'ALL_APE_CI': [0.375, 0.984375, 1.0, 1.0],
  'ALL_MAPE': 1.316,
  'ALL_RVALUE': 1.0,
  'APE_CI_THRESHOLDS': [1, 3, 5, 10],
  'APE_EXTREMES_ALL': {'MAX_APE': 3.07932, 'MIN_APE': 0.03036},
  'APE_EXTREMES_TEST': {'MAX_APE': 2.97073, 'MIN_APE': 0.03901},
  'APE_EXTREMES_TRAIN': {'MAX_APE': 3.07932, 'MIN_APE': 0.03036},
  'TEST_APE_CI': [0.4375, 1.0, 1.0, 1.0],
  'TEST_MAPE': 1.247,
  'TEST_MSE': 328.8784375000009,
  'TEST_MSE_PUH': 4.9560698784722015e-06,
  'TEST_RVALUE': 1.0,
  'TRAIN_APE_CI': [0.3125, 0.96875, 1.0, 1.0],
  'TRAIN_MAPE': 1.384,
  'TRAIN_RVALUE': 1.0},
 'evals': {'ME': {'ALL_APE_CI': [0.375, 0.984375, 1.0, 1.0],
   'ALL_MAPE': 1.316,
   'ALL_RVALUE': 1.0,
   'APE_CI_THRESHOLDS': [1, 3, 5, 10],
   'APE_EXTREMES_ALL': {'MAX_APE': 3.07932, 'MIN_APE': 0.03036},
   'APE_EXTREMES_TEST': {'MAX_APE': 2.97073, 'MIN_APE': 0.03901},
   'APE_EXTREMES_TRAIN': {'MAX_APE': 3.07932, 'MIN_APE': 0.03036},
   'TEST_APE_CI': [0.4375, 1.0, 1.0, 1.0],
   'TEST_MAPE': 1.247,
   'TEST_MS

Saving model...  D:\GithubRepos\CCMT-NILM\code\results\System_CVS\modeling\models\costmodel_ProcTime_FromMeter.pickle
Saving:  D:\GithubRepos\CCMT-NILM\code\results\System_CVS\modeling\models\costmodel_ProcTime_FromMeter_eval.JSON


### Cost: NILM Processing Energy

In [12]:
# building cost model for processing time
target = 'ProcEnergy_FromMeter'
features = ObjModel.features

### Hyperparameter Tuning

In [15]:
#  update the rf model parameters in the setting.JSON once the best paramters is identified

# ObjMLModel = model.PredictiveModel(models_path=ObjModel.models_path, dataset=ObjModel.dataset, features=ObjModel.features, target=target)
# param_grid = {
#                 'n_estimators': np.arange(4, 10, 2),
#                 'max_depth': np.arange(2, 10, 2),
#                 # 'min_samples_leaf': np.arange(1, 5, 1),
#                 # 'min_samples_split': np.arange(1, 5, 1),          
#                 # 'max_leaf_nodes': list(np.arange(10, 20, 2)) + [None],
#                 # 'bootstrap': [True, False],
#                 # "max_features":[0.5, 1]
#              }

# ObjMLModel.ParameterTuneGridCV(param_grid=param_grid, n_jobs=4)

#### Model building

In [16]:
print("Model building for {}...".format(target))
ObjCostModels_PE = ObjModel.ModelBuilding(features, target, isfigshow=True)

Model building for ProcEnergy_FromMeter...

Mean Estimator (ME) Model Fitting ...

features:  ['Number of Processes']
target:  ProcEnergy_FromMeter

****************************************
TEST_MSE_PUH: 1.1620597662955745e-08
TEST_MSE: 0.7045234572677097
ALL_MAPE: 1.945
TRAIN_MAPE: 1.122
TEST_MAPE: 2.767
ALL_RVALUE: 1.0
TRAIN_RVALUE: 1.0
TEST_RVALUE: 1.0
ALL_APE_CI: [0.359375, 0.8125, 0.90625, 1.0]
APE_EXTREMES_ALL: {'MIN_APE': 0.03229, 'MAX_APE': 6.34812}
TRAIN_APE_CI: [0.53125, 0.96875, 0.96875, 1.0]
APE_EXTREMES_TRAIN: {'MIN_APE': 0.03229, 'MAX_APE': 5.59161}
TEST_APE_CI: [0.1875, 0.65625, 0.84375, 1.0]
APE_EXTREMES_TEST: {'MIN_APE': 0.53545, 'MAX_APE': 6.34812}
APE_CI_THRESHOLDS: [1, 3, 5, 10]

****************************************

RandomForest Model Training ...

features:  ['Number of Processes', 'num_houses_inv']
target:  ProcEnergy_FromMeter
train seed:  223

****************************************
TEST_MSE_PUH: 1.074291132276473e-08
TEST_MSE: 0.64183173487604
ALL_MAPE: 1

### Save Model

In [17]:
ObjModel.ObjMLModel.costmodels
ObjModel.SaveTrainedModel(ObjCostModels_PE)

{'ME': {'ALL_APE_CI': [0.359375, 0.8125, 0.90625, 1.0],
  'ALL_MAPE': 1.945,
  'ALL_RVALUE': 1.0,
  'APE_CI_THRESHOLDS': [1, 3, 5, 10],
  'APE_EXTREMES_ALL': {'MAX_APE': 6.34812, 'MIN_APE': 0.03229},
  'APE_EXTREMES_TEST': {'MAX_APE': 6.34812, 'MIN_APE': 0.53545},
  'APE_EXTREMES_TRAIN': {'MAX_APE': 5.59161, 'MIN_APE': 0.03229},
  'TEST_APE_CI': [0.1875, 0.65625, 0.84375, 1.0],
  'TEST_MAPE': 2.767,
  'TEST_MSE': 0.7045234572677097,
  'TEST_MSE_PUH': 1.1620597662955745e-08,
  'TEST_RVALUE': 1.0,
  'TRAIN_APE_CI': [0.53125, 0.96875, 0.96875, 1.0],
  'TRAIN_MAPE': 1.122,
  'TRAIN_RVALUE': 1.0},
 'evals': {'ME': {'ALL_APE_CI': [0.359375, 0.8125, 0.90625, 1.0],
   'ALL_MAPE': 1.945,
   'ALL_RVALUE': 1.0,
   'APE_CI_THRESHOLDS': [1, 3, 5, 10],
   'APE_EXTREMES_ALL': {'MAX_APE': 6.34812, 'MIN_APE': 0.03229},
   'APE_EXTREMES_TEST': {'MAX_APE': 6.34812, 'MIN_APE': 0.53545},
   'APE_EXTREMES_TRAIN': {'MAX_APE': 5.59161, 'MIN_APE': 0.03229},
   'TEST_APE_CI': [0.1875, 0.65625, 0.84375, 1.0],
  

Saving model...  D:\GithubRepos\CCMT-NILM\code\results\System_CVS\modeling\models\costmodel_ProcEnergy_FromMeter.pickle
Saving:  D:\GithubRepos\CCMT-NILM\code\results\System_CVS\modeling\models\costmodel_ProcEnergy_FromMeter_eval.JSON
